# Spark SQL Basic
This is a short tutorial of Spark SQL and Spark DataFrames, including:
- DataFrame: creation, transformation and action methods
- Functions

# DataFrame

## Create a SparkSession

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

spark

In [2]:
sc = spark.sparkContext

sc

<SparkContext master=local[*] appName=Python Spark SQL basic example>

## Create a Dataframe

### From list

In [3]:
d = [('Chris', 'Berliner', 5), ('Matt', 'Yuengling', 10)]
df = spark.createDataFrame(d, ['drinker', 'beer', 'score'])
df.collect()

[Row(drinker='Chris', beer='Berliner', score=5),
 Row(drinker='Matt', beer='Yuengling', score=10)]

### From RDD

In [4]:
rdd = sc.parallelize(d)
df = spark.createDataFrame(rdd)
df.collect()
df = rdd.toDF()
df.collect()

[Row(_1='Chris', _2='Berliner', _3=5), Row(_1='Matt', _2='Yuengling', _3=10)]

In [5]:
df = spark.createDataFrame(rdd, ['drinker', 'beer', 'score'])
df.collect()

[Row(drinker='Chris', beer='Berliner', score=5),
 Row(drinker='Matt', beer='Yuengling', score=10)]

### From schema and column names

In [6]:
from pyspark.sql.types import *
schema = StructType([
    StructField("drinker", StringType(), True),
    StructField("beer", StringType(), True),
    StructField("score", IntegerType(), True)])
df = spark.createDataFrame(rdd, schema)
df.collect()

[Row(drinker='Chris', beer='Berliner', score=5),
 Row(drinker='Matt', beer='Yuengling', score=10)]

### From data sources
https://spark.apache.org/docs/latest/sql-data-sources.html

#### CSV

In [7]:
path = "path-to-people.csv"

In [8]:
# Spark SQL provides spark.read().csv("file_name") to read a file or directory of files in CSV format into Spark DataFrame
df = spark.read.csv(path)
df.show()

+------------------+
|               _c0|
+------------------+
|      name;age;job|
|Jorge;30;Developer|
|  Bob;32;Developer|
+------------------+



In [9]:
# Read a csv with delimiter, the default delimiter is ","
df = spark.read.option("delimiter", ";").csv(path)
df.show()

+-----+---+---------+
|  _c0|_c1|      _c2|
+-----+---+---------+
| name|age|      job|
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [10]:
# Read a csv with delimiter and a header
df = spark.read.option("delimiter", ";").option("header", True).csv(path)
df.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



In [11]:
# You can also use options() to use multiple options
df = spark.read.options(delimiter=";", header=True).csv(path)
df.show()
df.printSchema()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)



#### JSON

In [12]:
path = "path-to-people.json"

In [13]:
# Spark SQL provides spark.read().json("file_name") to read a file or directory of files in JSON format into Spark DataFrame
# It can automatically infer the schema of a JSON dataset
df = spark.read.json(path)
df.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [14]:
## The inferred schema can be visualized using the printSchema() method
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [15]:
# Alternatively, a DataFrame can be created for a JSON dataset represented by
# an RDD[String] storing one JSON object per string
jsonStrings = ['{"name":"Yin","address":{"city":"Columbus","state":"Ohio"}}']
rdd = sc.parallelize(jsonStrings)
df = spark.read.json(rdd)
df.show()

+----------------+----+
|         address|name|
+----------------+----+
|{Columbus, Ohio}| Yin|
+----------------+----+



### sql

In [16]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

In [17]:
sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

+----------------+----+
|         address|name|
+----------------+----+
|{Columbus, Ohio}| Yin|
+----------------+----+



In [18]:
# Temporary views in Spark SQL are session-scoped and will disappear if the session that creates it terminates. 
# If you want to have a temporary view that is shared among all sessions and keep alive until the Spark application terminates, 
# you can create a global temporary view. Global temporary view is tied to a system preserved database global_temp, 
# and we must use the qualified name to refer it, e.g. SELECT * FROM global_temp.view1.

# Register the DataFrame as a global temporary view
df.createGlobalTempView("other_people")

# Global temporary view is tied to a system preserved database `global_temp`
spark.sql("SELECT * FROM global_temp.other_people").show()

+----------------+----+
|         address|name|
+----------------+----+
|{Columbus, Ohio}| Yin|
+----------------+----+



In [19]:
# Global temporary view is cross-session
spark.newSession().sql("SELECT * FROM global_temp.other_people").show()

+----------------+----+
|         address|name|
+----------------+----+
|{Columbus, Ohio}| Yin|
+----------------+----+



## DataFrame Transformation Methods

### select(*cols)
Projects a set of expressions and returns a new DataFrame

In [20]:
df = spark.createDataFrame([(2, "Alice"), (5, "Bob")], schema=["age", "name"])

In [21]:
# Select all columns in the DataFrame
df.select('*').show()

+---+-----+
|age| name|
+---+-----+
|  2|Alice|
|  5|  Bob|
+---+-----+



In [22]:
# Select a column with other expressions in the DataFrame
df.select(df.name, (df.age + 10).alias('age')).show()

+-----+---+
| name|age|
+-----+---+
|Alice| 12|
|  Bob| 15|
+-----+---+



### selectExpr(*expr)
Projects a set of SQL expressions and returns a new DataFrame

In [23]:
# This is a variant of select() that accepts SQL expressions
df.selectExpr("age * 2 as double_age", "abs(age)").show()

+----------+--------+
|double_age|abs(age)|
+----------+--------+
|         4|       2|
|        10|       5|
+----------+--------+



### filter(condition)
Filters rows using the given condition

In [24]:
# Filter by Column instances
df.filter(df.age > 3).show()

+---+----+
|age|name|
+---+----+
|  5| Bob|
+---+----+



In [25]:
# where() is an alias for filter()

In [26]:
df.where(df.age == 2).show()

+---+-----+
|age| name|
+---+-----+
|  2|Alice|
+---+-----+



In [27]:
# Filter by SQL expression in a string

In [28]:
df.filter("age > 3").show()

+---+----+
|age|name|
+---+----+
|  5| Bob|
+---+----+



### drop(*cols)
Returns a new DataFrame without specified columns

In [29]:
df.drop('age').show()

+-----+
| name|
+-----+
|Alice|
|  Bob|
+-----+



In [30]:
df.drop(df.age).show()

+-----+
| name|
+-----+
|Alice|
|  Bob|
+-----+



### groupBy(*cols)
Groups the DataFrame using the specified columns, so we can run aggregation on them

In [31]:
df = spark.createDataFrame([(2, "Alice"), (2, "Bob"), (2, "Bob"), (5, "Bob")], schema=["age", "name"])

In [32]:
# Group-by ‘name’, and specify a dictionary to calculate the summation of ‘age’
df.groupBy("name").agg({"age": "sum"}).sort("name").show()

+-----+--------+
| name|sum(age)|
+-----+--------+
|Alice|       2|
|  Bob|       9|
+-----+--------+



In [33]:
# Group-by ‘name’, and calculate maximum values
df.groupBy(df.name).max().sort("name").show()

+-----+--------+
| name|max(age)|
+-----+--------+
|Alice|       2|
|  Bob|       5|
+-----+--------+



### join(other[, on, how])
Joins with another DataFrame, using the given join expression
https://images.datacamp.com/image/upload/v1679944054/Marketing/Blog/Joining_Data_in_SQL_Cheat_Sheet.pdf

In [34]:
from pyspark.sql import Row
from pyspark.sql.functions import desc
df = spark.createDataFrame([(2, "Alice"), (5, "Bob")]).toDF("age", "name")
df2 = spark.createDataFrame([Row(height=80, name="Tom"), Row(height=85, name="Bob")])
df3 = spark.createDataFrame([Row(age=2, name="Alice"), Row(age=5, name="Bob")])
df4 = spark.createDataFrame([
    Row(age=10, height=80, name="Alice"),
    Row(age=5, height=None, name="Bob"),
    Row(age=None, height=None, name="Tom"),
    Row(age=None, height=None, name=None),
])

In [35]:
# Inner join on columns
df.join(df2, 'name').select(df.name, df2.height).show()

+----+------+
|name|height|
+----+------+
| Bob|    85|
+----+------+



In [36]:
# Outer join for both DataFrames on the ‘name’ column
df.join(df2, 'name', 'outer').select('name', 'height').sort(desc("name")).show()
df.join(df2, df.name == df2.name, 'outer').select(
    df.name, df2.height).sort(desc("name")).show()

+-----+------+
| name|height|
+-----+------+
|  Tom|    80|
|  Bob|    85|
|Alice|  NULL|
+-----+------+

+-----+------+
| name|height|
+-----+------+
|  Bob|    85|
|Alice|  NULL|
| NULL|    80|
+-----+------+



### withColumn(colName, col)
Returns a new DataFrame by adding a column or replacing the existing column that has the same name

In [37]:
df = spark.createDataFrame([(2, "Alice"), (5, "Bob")], schema=["age", "name"])
df.withColumn('age2', df.age + 2).show()

+---+-----+----+
|age| name|age2|
+---+-----+----+
|  2|Alice|   4|
|  5|  Bob|   7|
+---+-----+----+



### agg(*exprs)
Aggregate on the entire DataFrame without groups (shorthand for df.groupBy().agg())

Available predefined aggregate functions:

- avg(*cols) Computes average values for each numeric columns for each group.
- count() Counts the number of records for each group.
- max(*cols) Computes the max value for each numeric columns for each group.
- mean(*cols) Computes average values for each numeric columns for each group.
- min(*cols) Computes the min value for each numeric column for each group.
- sum(*cols) Computes the sum for each numeric columns for each group

In [38]:
from pyspark.sql import functions as func
df = spark.createDataFrame([(2, "Alice"), (5, "Bob")], schema=["age", "name"])
df.agg({"age": "max"}).show()
df.agg(func.max("age")).show()
df.agg(func.max(df.age)).show()

+--------+
|max(age)|
+--------+
|       5|
+--------+

+--------+
|max(age)|
+--------+
|       5|
+--------+

+--------+
|max(age)|
+--------+
|       5|
+--------+



### orderBy(*cols, **kwargs)
Returns a new DataFrame sorted by the specified column(s)

In [39]:
from pyspark.sql.functions import desc, asc
df = spark.createDataFrame([
    (2, "Alice"), (5, "Bob")], schema=["age", "name"])

In [40]:
# Sort the DataFrame in descending order
df.orderBy(df.age.desc()).show()
df.orderBy(desc("age"), "name").show()

+---+-----+
|age| name|
+---+-----+
|  5|  Bob|
|  2|Alice|
+---+-----+

+---+-----+
|age| name|
+---+-----+
|  5|  Bob|
|  2|Alice|
+---+-----+



## DataFrame Action Methods
- show([n, truncate, vertical]): Prints the first n rows to the console
- count(): Returns the number of rows in this DataFrame
- collect(): Returns all the records as a list of Row

## DataFrame Attributes

### columns
Retrieves the names of all columns in the DataFrame as a list

In [41]:
df = spark.createDataFrame(
    [(14, "Tom", "CA"), (23, "Alice", "NY"), (16, "Bob", "TX")],
    ["age", "name", "state"]
)
df.columns

['age', 'name', 'state']

### rdd
Returns the content as an pyspark.RDD of Row

In [42]:
df = spark.range(1)
type(df.rdd)

pyspark.rdd.RDD

### schema
Returns the schema of this DataFrame as a pyspark.sql.types.StructType

In [43]:
df = spark.createDataFrame([(14, "Tom"), (23, "Alice"), (16, "Bob")], ["age", "name"])

In [44]:
# Retrieve the schema of the current DataFrame
df.schema

StructType([StructField('age', LongType(), True), StructField('name', StringType(), True)])

# Functions

## lit(col)
Creates a Column of literal value

In [45]:
from pyspark.sql.functions import lit
df = spark.range(2)
df.select(lit(5).alias('height'), df.id).show()

+------+---+
|height| id|
+------+---+
|     5|  0|
|     5|  1|
+------+---+



## col
Returns a Column based on the given column name

In [46]:
from pyspark.sql.functions import col
df.select(col('id')).show()

+---+
| id|
+---+
|  0|
|  1|
+---+



## concat(*cols)
Concatenates multiple input columns together into a single column

In [47]:
from pyspark.sql.functions import concat, col
df = spark.createDataFrame([('abcd','123')], ['s', 'd'])
df.select(concat(df.s, df.d).alias('sd')).show()
df.select(concat(col('s'), col('d')).alias('sd')).show()

+-------+
|     sd|
+-------+
|abcd123|
+-------+

+-------+
|     sd|
+-------+
|abcd123|
+-------+



## udf([f, returnType, useArrow])
Creates a user defined function (UDF)

In [48]:
from pyspark.sql.types import StringType 
from pyspark.sql.functions import udf
maturity_udf = udf(lambda age: "adult" if age >=18 else "child", StringType())
df = spark.createDataFrame([{'name': 'Alice', 'age': 1}]) 
df.withColumn("maturity", maturity_udf(df.age)).show()

+---+-----+--------+
|age| name|maturity|
+---+-----+--------+
|  1|Alice|   child|
+---+-----+--------+



# Stop SparkSession

In [49]:
spark.stop()